In [1]:
import pandas as pd
import os
from zipfile import ZipFile 
import re
import collections
import numpy as np
import xmltodict
import gzip

### Disgenet

In [11]:
disgenet_path = os.path.join('../data/raw', 'curated_gene_disease_associations.tsv.gz')
with gzip.open(disgenet_path) as file:
    disgenet = pd.read_csv(file, sep='\t', engine='python')

In [12]:
disgenet = disgenet[['geneSymbol', 'diseaseId', 'diseaseSemanticType']]

In [13]:
disgenet

,geneSymbol,diseaseId,diseaseSemanticType
0,A1BG,C0019209,Finding
1,A1BG,C0036341,Mental or Behavioral Dysfunction
2,A2M,C0002395,Disease or Syndrome
3,A2M,C0007102,Neoplastic Process
4,A2M,C0009375,Neoplastic Process
...,...,...,...
81741,HBB-LCR,C0002875,Disease or Syndrome
81742,HBB-LCR,C0005283,Disease or Syndrome
81743,HBB-LCR,C0019025,Disease or Syndrome
81744,HBB-LCR,C0085578,Disease or Syndrome


### SIDER

In [25]:
sider_path = os.path.join('../data/raw', 'meddra_all_se.tsv.gz')
with gzip.open(sider_path) as file:
    sider = pd.read_csv(file, sep='\t', engine='python',header=None)

In [26]:
sider = sider[[0, 1, 5]]
sider.columns = ['drug1', 'drug2', 'side_effect']

In [27]:
sider

,drug1,drug2,side_effect
0,CID100000085,CID000010917,Abdominal cramps
1,CID100000085,CID000010917,Abdominal pain
2,CID100000085,CID000010917,Abdominal pain
3,CID100000085,CID000010917,Gastrointestinal pain
4,CID100000085,CID000010917,Abdominal pain
...,...,...,...
309844,CID171306834,CID071306834,Respiratory failure
309845,CID171306834,CID071306834,Abnormal vision
309846,CID171306834,CID071306834,Visual impairment
309847,CID171306834,CID071306834,Warts
